# S&P500 historical 20year data processing 
### In this file we are performing data preparation for training and test datasets for algo trading models.

In [121]:
# Imports
import pandas as pd
from pathlib import Path

from pandas.tseries.offsets import DateOffset

In [122]:
# Import the sp500 dataset into a Pandas Dataframe
trading_df = pd.read_csv(
    Path("../data/SP500_Data.csv"), 
    index_col="Date",
    infer_datetime_format=True, 
    parse_dates=True
    
)
# Review the DataFrame
trading_df.head()

,Price,Open,High,Low,Change %
Date,,,,,
2002-07-03,954.0,948.1,954.3,934.9,0.62%
2002-07-05,989.0,965.8,989.1,954.0,3.67%
2002-07-08,977.0,989.0,993.6,972.9,-1.21%
2002-07-09,952.8,977.0,979.6,951.7,-2.48%
2002-07-10,920.5,952.8,956.3,920.3,-3.39%


In [123]:
trading_df.shape

(5000, 5)

In [124]:
# Calculate the daily returns using the closing prices and the pct_change function
trading_df["actual_returns"] = trading_df["Price"].pct_change()

# Drop all NaN values from the DataFrame
trading_df = trading_df.dropna()

# Review the DataFrame
display(trading_df.head())
display(trading_df.tail())

,Price,Open,High,Low,Change %,actual_returns
Date,,,,,,
2002-07-05,989.0,965.8,989.1,954.0,3.67%,0.036688
2002-07-08,977.0,989.0,993.6,972.9,-1.21%,-0.012133
2002-07-09,952.8,977.0,979.6,951.7,-2.48%,-0.024770
2002-07-10,920.5,952.8,956.3,920.3,-3.39%,-0.033900
2002-07-11,927.4,920.5,929.2,900.9,0.75%,0.007496


,Price,Open,High,Low,Change %,actual_returns
Date,,,,,,
2022-05-05,4152.38,4255.50,4256.39,4105.51,-3.44%,-0.034368
2022-05-06,4123.34,4128.17,4157.69,4067.91,-0.70%,-0.006994
2022-05-09,3991.24,4081.27,4081.27,3975.48,-3.20%,-0.032037
2022-05-10,4001.05,4035.18,4068.82,3958.17,0.25%,0.002458
2022-05-11,3935.18,3990.08,4049.09,3928.82,-1.65%,-0.016463


In [125]:
# Define a window size of 4
short_window = 12

# Create a simple moving average (SMA) using the short_window and assign this to a new columns called sma_fast
trading_df["sma_fast"] = trading_df["Price"].rolling(window=short_window).mean()

In [126]:
# Define a window size of 50
long_window = 26

# Create a simple moving average (SMA) using the long_window and assign this to a new columns called sma_slow
trading_df["sma_slow"] = trading_df["Price"].rolling(window=long_window).mean()

In [127]:
trading_df.head()

,Price,Open,High,Low,Change %,actual_returns,sma_fast,sma_slow
Date,,,,,,,,
2002-07-05,989.0,965.8,989.1,954.0,3.67%,0.036688,NaN,NaN
2002-07-08,977.0,989.0,993.6,972.9,-1.21%,-0.012133,NaN,NaN
2002-07-09,952.8,977.0,979.6,951.7,-2.48%,-0.024770,NaN,NaN
2002-07-10,920.5,952.8,956.3,920.3,-3.39%,-0.033900,NaN,NaN
2002-07-11,927.4,920.5,929.2,900.9,0.75%,0.007496,NaN,NaN


In [128]:
# Drop the NaNs using dropna()
trading_df = trading_df.dropna()

In [129]:
# Assign a copy of the sma_fast and sma_slow columns to a new DataFrame called X
X = trading_df[["sma_fast", "sma_slow"]].copy()

# Display sample data
display(X.head())
display(X.tail())

,sma_fast,sma_slow
Date,,
2002-08-09,878.241667,890.088462
2002-08-12,883.666667,886.811538
2002-08-13,886.283333,883.242308
2002-08-14,888.000000,881.965385
2002-08-15,890.283333,882.338462


,sma_fast,sma_slow
Date,,
2022-05-05,4248.584167,4374.699615
2022-05-06,4220.575000,4356.272308
2022-05-09,4187.040000,4335.535000
2022-05-10,4164.479167,4314.580769
2022-05-11,4134.400833,4289.678462


In [130]:
## Create the target set

# Create a new column in the trading_df called signal setting its value to zero.
trading_df["signal"] = 0.0

# Create the signal to buy
trading_df.loc[(trading_df["actual_returns"] >= 0), "signal"] = 1

# Create the signal to sell
trading_df.loc[(trading_df["actual_returns"] < 0), "signal"] = -1

# Copy the new signal column to a new Series called y.
y = trading_df["signal"].copy()

## Creating the Training Datasets

In [131]:
# Select the start of the training period
training_begin = X.index.min()

# Display the training begin date
print(training_begin)

# Select the ending period for the training data with an offset of 4 years
training_end = X.index.min() + DateOffset(years=4)

# Display the training end date
print(training_end)

2002-08-09 00:00:00
2006-08-09 00:00:00


In [132]:
# Generate the X_train and y_train DataFrames
X_long_train = X.loc[training_begin:training_end]
y_long_train = y.loc[training_begin:training_end]

# Display sample data
display(X_long_train.head())
display(X_long_train.tail())

,sma_fast,sma_slow
Date,,
2002-08-09,878.241667,890.088462
2002-08-12,883.666667,886.811538
2002-08-13,886.283333,883.242308
2002-08-14,888.000000,881.965385
2002-08-15,890.283333,882.338462


,sma_fast,sma_slow
Date,,
2006-08-03,1266.202500,1262.398462
2006-08-04,1267.831667,1263.681538
2006-08-07,1270.051667,1263.793077
2006-08-08,1272.650833,1263.842308
2006-08-09,1273.070833,1263.294615


In [133]:
display(y_long_train.head())
display(y_long_train.tail())

Date
2002-08-09    1.0
2002-08-12   -1.0
2002-08-13   -1.0
2002-08-14    1.0
2002-08-15    1.0
Name: signal, dtype: float64

Date
2006-08-03    1.0
2006-08-04   -1.0
2006-08-07   -1.0
2006-08-08   -1.0
2006-08-09   -1.0
Name: signal, dtype: float64

## Creating the long term testing Datasets

In [134]:
# Generate the X_long_test and y_long_test DataFrames

X_long_test = X.loc[training_end:]
y_long_test = y.loc[training_end:]

# Display sample data
# Display sample data
display(X_long_test.head())
display(X_long_test.tail())

,sma_fast,sma_slow
Date,,
2006-08-09,1273.070833,1263.294615
2006-08-10,1273.315000,1263.329231
2006-08-11,1273.176667,1263.046923
2006-08-14,1273.594167,1263.151923
2006-08-15,1274.180000,1263.853462


,sma_fast,sma_slow
Date,,
2022-05-05,4248.584167,4374.699615
2022-05-06,4220.575000,4356.272308
2022-05-09,4187.040000,4335.535000
2022-05-10,4164.479167,4314.580769
2022-05-11,4134.400833,4289.678462


In [135]:
display(y_long_test.head())
display(y_long_test.tail())

Date
2006-08-09   -1.0
2006-08-10    1.0
2006-08-11   -1.0
2006-08-14    1.0
2006-08-15    1.0
Name: signal, dtype: float64

Date
2022-05-05   -1.0
2022-05-06   -1.0
2022-05-09   -1.0
2022-05-10    1.0
2022-05-11   -1.0
Name: signal, dtype: float64

### Generate the 5 years data X_short_test and y_short_test DataFrames

In [136]:
short_testing_begin = X.index.max() - DateOffset(years=5)
short_testing_begin

Timestamp('2017-05-11 00:00:00')

In [137]:
X_short_test = X.loc[short_testing_begin:]
y_short_test = y.loc[short_testing_begin:]

# Display sample data
display(X_short_test.head())
display(X_short_test.tail())
display(y_short_test.head())
display(y_short_test.tail())

,sma_fast,sma_slow
Date,,
2017-05-11,2392.269167,2371.334231
2017-05-12,2392.556667,2372.793846
2017-05-15,2393.685833,2374.518077
2017-05-16,2395.058333,2376.253846
2017-05-17,2392.450000,2376.248846


,sma_fast,sma_slow
Date,,
2022-05-05,4248.584167,4374.699615
2022-05-06,4220.575000,4356.272308
2022-05-09,4187.040000,4335.535000
2022-05-10,4164.479167,4314.580769
2022-05-11,4134.400833,4289.678462


Date
2017-05-11   -1.0
2017-05-12   -1.0
2017-05-15    1.0
2017-05-16   -1.0
2017-05-17   -1.0
Name: signal, dtype: float64

Date
2022-05-05   -1.0
2022-05-06   -1.0
2022-05-09   -1.0
2022-05-10    1.0
2022-05-11   -1.0
Name: signal, dtype: float64

In [138]:
# Generate the X_short_train and y_short_train DataFrames for the previous 3 years of short testing dataset
short_testing_end = short_testing_begin
short_testing_begin =  short_testing_end - DateOffset(years=2)

In [139]:
X_short_train = X.loc[short_testing_begin:short_testing_end]
y_short_train = y.loc[short_testing_begin:short_testing_end]

# Display sample data
display(X_short_train.head())
display(y_short_train.head())
display(X_short_train.tail())
display(y_short_train.tail())

,sma_fast,sma_slow
Date,,
2015-05-11,2102.962500,2098.741923
2015-05-12,2101.415000,2099.453462
2015-05-13,2100.545000,2100.305385
2015-05-14,2101.073333,2101.813077
2015-05-15,2102.396667,2103.026538


Date
2015-05-11   -1.0
2015-05-12   -1.0
2015-05-13   -1.0
2015-05-14    1.0
2015-05-15    1.0
Name: signal, dtype: float64

,sma_fast,sma_slow
Date,,
2017-05-05,2382.012500,2365.926923
2017-05-08,2385.640833,2367.131538
2017-05-09,2389.660000,2368.446923
2017-05-10,2391.783333,2370.015769
2017-05-11,2392.269167,2371.334231


Date
2017-05-05    1.0
2017-05-08    1.0
2017-05-09   -1.0
2017-05-10    1.0
2017-05-11   -1.0
Name: signal, dtype: float64

## Create train and test data files

In [140]:
X_long_train.to_csv("../data/X_long_train.csv")
y_long_train.to_csv("../data/y_long_train.csv")

X_long_test.to_csv("../data/X_long_test.csv")
y_long_test.to_csv("../data/y_long_test.csv")

X_short_train.to_csv("../data/X_short_train.csv")
y_short_train.to_csv("../data/y_short_train.csv")

X_short_test.to_csv("../data/X_short_test.csv")
y_short_test.to_csv("../data/y_short_test.csv")